A megoldásomhoz ingyenes elemek felhasználásával próbáltam eljutni:
https://huggingface.co/docs/transformers/model_doc/rag

# **Első Próba**

Itt próbáltam kikésrletezni, mely parancsokkal képes meghívni a facebook/ rag tokenjeihez kapcsolódó modulokat, a meghívásokat mindenképpen futtatni kell

In [ ]:
%pip install datasets

In [ ]:
!apt install libomp-dev
!pip install faiss-cpu

In [ ]:
import datasets
import faiss

In [ ]:
from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration
from transformers import pipeline

In [ ]:
# Tokenizáló betöltése
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-base")

In [ ]:
# Retriever betöltése
retriever = RagRetriever.from_pretrained("facebook/rag-token-base", index_name="exact", use_dummy_dataset=True)

A következő része a modell *maga* mivel a modellt rendkívül sokáig tartott tervezetten megfuttatni, ezért lett két részre szedve


In [ ]:
from transformers import RagModel, RagSequenceForGeneration

In [ ]:
# Answerer betöltése
model = RagTokenForGeneration.from_pretrained("facebook/rag-token-base")
#model = RagModel.from_pretrained("facebook/rag-token-base", retriever=retriever)


A RagModel-t nem enged kérdés válaszra használni ezt az alábbi kód futtatásából tudtam meg, Rag sq generationt sem lehet se pipelineba se text2text-generation pipelineba rakni

In [ ]:
model = RagSequenceForGeneration.from_pretrained("facebook/rag-token-nq", retriever=retriever)

In [ ]:
# Pipeline létrehozása
qa_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)
# Kérdés
question = "What is the name of the main square of Budapest?"

# Pipeline futtatása
result = qa_pipeline(question=question, context="Hero square is the main square of Budapest")

# Eredmény kiíratása
print(result)

# **Második Próba**

Kiderült, hogy nem képes a beírt tokenizert rendesen exportálni, így nagyon tesztfuttatni sem lehet a működést, nem hogy integrálni

In [ ]:
from transformers import AutoTokenizer, RagRetriever, RagSequenceForGeneration, RagTokenizer

import torch

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained("facebook/rag-sequence-nq")
#tokenizer = AutoTokenizer.from_pretrained("facebook/rag-token-base")
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-base")

#BartTokenizerFast-ot tud csak futtatni

In [ ]:
#retriever = RagRetriever.from_pretrained("facebook/rag-sequence-nq", index_name="exact", use_dummy_dataset=True)
retriever = RagRetriever.from_pretrained( "facebook/rag-token-base", index_name="exact", use_dummy_dataset=True)

In [ ]:
# initialize with RagRetriever to do everything in one forward call
#model = RagSequenceForGeneration.from_pretrained("facebook/rag-token-nq", retriever=retriever)
model = RagSequenceForGeneration.from_pretrained("facebook/rag-token-base", retriever=retriever)


In [ ]:
inputs = tokenizer("How many people live in Paris?", return_tensors="pt")

targets = tokenizer(text_target="In Paris, there are 10 million people.", return_tensors="pt")

input_ids = inputs["input_ids"]

labels = targets["input_ids"]

outputs = model(input_ids=input_ids, labels=labels)

In [ ]:
print(outputs)

In [ ]:
output_ids = outputs.logits.argmax(-1)
output_text = tokenizer.decode(output_ids[0])
print(output_text)

Ci.. to proportional how::::
